In [40]:
# 방법2. json to dataframe
import pandas as pd
resourceDf = pd.read_json("../data/resource.json") 
applicationDf = pd.read_json("../data/application.json")
paDf = pd.read_json("../data/productApplication.json")
serverDf = pd.read_json("../data/server.json")

serverResourceDf = resourceDf[
    resourceDf["type"].isin(['server','legacy-server','legacy-physical-server'])]

total_resource_count = serverResourceDf.shape[0]
total_server_count =serverDf.shape[0]
serverResourceDf.shape


(134009, 10)

In [42]:
import json

defaultPA = json.loads(paDf[paDf["name"]=='default'].to_json())
default_pa_id = defaultPA['_id']['0']['$oid']

print(default_pa_id)

61d3cd987d7ab1f2dc6bab0d


In [43]:
pd.set_option('mode.chained_assignment',  None) 

applicationDf["parentGroup"] = applicationDf["parentGroup"].apply(lambda x : x['_id']['$oid'])
applicationDf["_id"] = applicationDf["_id"].apply(lambda x : x['$oid'])
serverResourceDf["parentGroups"]= serverResourceDf["parentGroups"].apply(lambda x : x[0]['_id']['$oid'])

In [44]:
lookupServerApplicationDf = pd.merge(left = serverResourceDf , right = applicationDf, left_on='parentGroups', right_on='_id', how = "left" ,suffixes=["_resource","_application"])

In [45]:
non_default_pa_server = lookupServerApplicationDf[lookupServerApplicationDf["parentGroup"] != default_pa_id]
non_default_pa_server.dropna(axis=0,how='any',subset=["parentGroup","parentGroups"],inplace = True)
registered_server_count= non_default_pa_server.shape[0]
print(registered_server_count)

75323


In [48]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,8))
registered_server_count= non_default_pa_server.shape[0]
unregistered_server_count=total_resource_count-registered_server_count
values1 = [registered_server_count, unregistered_server_count]
labels1 = ["registered server type resource", "unregistered server type resource"]
colors1 = sns.color_palette('hls',len(labels1))

plt.subplot(1,2,1)
result1 = plt.pie(values1,labels=labels1, autopct='%.3f%%', colors = colors1, shadow=True)

server_count = non_default_pa_server[non_default_pa_server["type"]=="server"].shape[0]
legacy_server_count= non_default_pa_server[non_default_pa_server["type"]=="legacy-server"].shape[0]
legacy_physical_server_count=non_default_pa_server[non_default_pa_server["type"]=="legacy-physical-server"].shape[0]

values2 = [server_count,legacy_server_count,legacy_physical_server_count ]
labels2 = ["server", "legacy_server", "legacy_physical_server"]
colors2 = sns.color_palette('hls',len(labels2))

plt.subplot(1,2,2)
result2 = plt.pie(values2,labels=labels2, autopct='%.3f%%', colors = colors2, shadow=True)

NameError: name 'labels' is not defined

<Figure size 720x576 with 0 Axes>

In [ ]:
plt.figure(figsize=(10,8))

values3 = [registered_server_count, total_server_count-registered_server_count]
labels3 = ["registered server type resource", "unregistered server"]
colors3 = sns.color_palette('hls',len(labels))
result3 = plt.pie(values3,labels=labels3, autopct='%.3f%%', colors = colors3, shadow=True)

In [47]:
from datetime import datetime

def save_portion_and_count_frame(result, labels, values, path, prefix):
    percentages=[]
    for res in result[2]:
        percentages.append(res.get_text())
    portion_df = pd.DataFrame({'Target':labels, 'Portion': percentages,'Count': values})
    portion_df.to_csv(path+prefix+"_"+str(datetime.now())+".csv")
    

path="./"
save_portion_and_count_frame(result1, labels1, values1, path, "registered server type resources on lai resource")
save_portion_and_count_frame(result2, labels2, values2, path, "registered resources per type")
save_portion_and_count_frame(result3, labels3, values3, path, "registered server type resources on servers")

NameError: name 'result1' is not defined

In [170]:
test_df =pd.read_csv("registered resources per type_2022-08-08 16:06:40.819073.csv")
test_df=test_df[test_df.columns[1:]]
test_df.head()

,Target,Portion,Count
0,server,82.596%,62214
1,legacy_server,2.050%,1544
2,legacy_physical_server,15.354%,11565


In [171]:
! git add .
! git commit -m "export report"
! git push origin master

[master 368d0cf] export report
 4 files changed, 129 insertions(+), 78 deletions(-)
 create mode 100644 registered resources per type_2022-08-08 16:06:40.819073.csv
 create mode 100644 registered server type resources on lai resource_2022-08-08 16:06:40.813933.csv
 create mode 100644 registered server type resources on servers_2022-08-08 16:06:40.821924.csv
Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 10 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 20.85 KiB | 10.43 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 1 local object.
To https://github.com/2jimoo/data_stastics
   a2d42cd..368d0cf  master -> master
